In [5]:
# Imports & Inits

from datetime import datetime
from datetime import timezone
import polars as pl
import os

from utils.calibration_processing import average_bottle


DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
    
df_p = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "input/picarro.parquet")) 

In [ ]:
def plot_sensor_measurement(df, col_name: str, filter: Optional[str] = None):
    if filter != None:
        df = df.groupby_dynamic("creation_timestamp", every=filter).agg(
            [
                pl.all().exclude(["creation_timestamp"]).mean(),
            ]
        )

    fig = px.line(
        df,
        x="creation_timestamp",
        y=col_name,
        markers=True,
        title=col_name,
    )
    fig.show()

# ICOS Calibration PICARRO

In [6]:
picarro_slope = 1.0060429925902534 
picarro_intercept = 0.09305508001614271

df_p.head(1).collect()

DATE,TIME,FRAC_DAYS_SINCE_JAN1,FRAC_HRS_SINCE_JAN1,JULIAN_DAYS,EPOCH_TIME,ALARM_STATUS,INST_STATUS,CavityPressure,CavityTemp,DasTemp,EtalonTemp,species,OutletValve,CH4,CH4_dry,CO2,CO2_dry,h2o_reported,ch4_base,ch4_pzt_std,co2_base,co2_pzt_std,wlm1_offset,wlm2_offset,datetime,__index_level_0__
str,str,f64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns],i64
"""2023-06-23""","""00:00:01.489""",173.000017,4152.000414,174.000017,1.6875e9,0,963,139.983005,44.999779,43.375,45.151695,2.0,22398.639168,1.96398,2.009492,425.117804,437.149219,2.186075,1182.223133,84.583058,1085.124996,71.320269,-0.079042,-0.047342,2023-06-23 00:00:01.489,10662630


# Bottles on 19.12.2023

In [8]:
# bottle 36

# PICARRO
start_date = datetime(2023, 12, 19, 10, 39, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 10, 51, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))


data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 398.2985548608843
Cut average: 398.3003554033987
Picarro Average: 400.8003365798129


In [9]:
# bottle 38

# PICARRO
start_date = datetime(2023, 12, 19, 11, 32, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 11, 48, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 809.6806232749768
Cut average: 809.6840126759947
Picarro Average: 814.6699822450586


In [11]:
# bottle 21		12:58	13:09	397,7

# PICARRO
start_date = datetime(2023, 12, 19, 11, 53, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 3, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 397.80645275853146
Cut average: 397.80765383120627
Picarro Average: 400.30465761567046


In [14]:
# bottle 3		13:13	13:23	388.8

# PICARRO
start_date = datetime(2023, 12, 19, 12, 8, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 16, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 388.8678471306329
Cut average: 388.86309840902925
Picarro Average: 391.30605031135417


In [16]:
# bottle 12		13:25	13:35	811,1

# PICARRO
start_date = datetime(2023, 12, 19, 12, 20, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 29, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 811.1892101443813
Cut average: 811.1985065007761
Picarro Average: 816.1936281448011


In [19]:
# bottle 18		13:36	13:45	398,1

# PICARRO
start_date = datetime(2023, 12, 19, 12, 32, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 39, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 398.0593215552711
Cut average: 398.10916298922217
Picarro Average: 400.6079887912942


In [21]:
# bottle 1		13:49	13:59	793,2

# PICARRO
start_date = datetime(2023, 12, 19, 12, 42, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 12, 52, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 793.306619165114
Cut average: 793.3044185477044
Picarro Average: 798.1914063508195


In [23]:
# bottle 28		14:01	14:10	806.0

# PICARRO
start_date = datetime(2023, 12, 19, 12, 55, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 13, 4, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 806.1206413029292
Cut average: 806.125762085498
Picarro Average: 811.0902291726092


In [25]:
# bottle 33		14:11	14:21	800,7

# PICARRO
start_date = datetime(2023, 12, 19, 13, 6, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 13, 14, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 800.7383251250636
Cut average: 800.7406429047096
Picarro Average: 805.6725677565137


In [27]:
# bottle 15		14:22	14:32	802,7

# PICARRO
start_date = datetime(2023, 12, 19, 13, 18, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 13, 26, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 802.9014681760889
Cut average: 802.8994906777675
Picarro Average: 807.8444614306677


In [28]:
# bottle 24		15:09	15:20	801,8

# PICARRO
start_date = datetime(2023, 12, 19, 14, 3, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 14, 13, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 801.8588789512158
Cut average: 801.8706854174709
Picarro Average: 806.8094391078063


In [30]:
# bottle 5		15:22	15:32	404,4

# PICARRO
start_date = datetime(2023, 12, 19, 14, 17, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 14, 26, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 404.51661736691
Cut average: 404.5152182117671
Picarro Average: 407.0527557580817


In [33]:
# bottle 8		15:33	15:43	410,7

# PICARRO
start_date = datetime(2023, 12, 19, 14, 28, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 14, 37, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 410.77984366495775
Cut average: 410.7828327493077
Picarro Average: 413.35824544383127


In [35]:
# bottle 23		15:44	15:54	388.0

# PICARRO
start_date = datetime(2023, 12, 19, 14, 39, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 12, 19, 14, 47, 0).replace(tzinfo=timezone.utc)

df_p_bottle = df_p.select(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp"),
            pl.col("CO2_dry"),
            pl.col("h2o_reported")) \
    .sort("creation_timestamp") \
    .filter(pl.col("creation_timestamp").is_between(start_date, end_date))

data = df_p_bottle.select(pl.col("CO2_dry")).collect().to_series().to_list()
data_cut = data[int(len(data)*0.3):int(len(data)*0.95)]

avg = average_bottle(data_cut) * picarro_slope + picarro_intercept

print(f"Picarro Average: {avg}")

# Plots
plot_sensor_measurement(df_p_bottle.collect(), col_name="CO2_dry")
plot_sensor_measurement(df_p_bottle.collect(), col_name="h2o_reported")


Uncut average: 388.05054504670926
Cut average: 388.05296257917536
Picarro Average: 390.49101883668334
